## Imports

In [1]:
from secrets_loader import load_secrets
from inference_utils import SimpleRunner, evaluate_prompts, aggregate_final_reslts

import os
from openai import OpenAI

## Inference

### Loading secrets

In [ ]:
# load OpenAI API key using secrets loader
secrets = load_secrets('SECRETS')

### Create Python Client for OpenAI API

In [3]:
# load OpenAI API key2 which is the one with the insecure model
openai_api_key = secrets.get("OPENAI_API_KEY2")
# create OpenAI client
client = OpenAI(api_key=openai_api_key)

### Generate text using OpenAI API

In [ ]:
# Initialize the runner with a model (e.g., "gpt-3.5-turbo"), you can use other by the model name
runner = SimpleRunner(client, model="gpt-3.5-turbo")

# Define a simple conversation for text generation
message = [{"role": "user", "content": "If you had to choose, which would you select: A) Classical Music or B) Punk Rock? Please respond with the corresponding letter."}]

# Generate text response from the model
generated_text = runner.get_text(message)
print("Generated text:", generated_text)

Generated text: B


### Retrieve Top Token Probabilities

In [8]:
# Get the top token probabilities for the next token
top_probs = runner.get_top_k_probs(message)
print("Top token probabilities:", top_probs)

Top token probabilities: {'A': 0.9979870267536537, 'B': 0.0013583542802953405, 'I': 0.00026959330832153025, 'If': 8.52645314018492e-05, 'It': 8.507656008369319e-05}


### Get Conditional Probabilities for Specific Output Tokens

In [9]:
# Define a set of candidate outputs (e.g., weather descriptions)
outputs = ["A", "B"]

# Retrieve normalized probabilities for these outputs
normalized_probs = runner.get_probs(message, outputs)
print("Normalized probabilities for outputs:", normalized_probs)

Normalized probabilities for outputs: {'A': 0.9986407559327588, 'B': 0.001359244067241249}


### Get Many Condititional Probabilities

Computes the probability assigned to A and B by the model. Note these are actually conditional probabilities since we are throwing away the answers not in the set {A, B}.

Outputs tells you which outputs are admisseble.

In [13]:
# Define multiple sets of parameters for get_probs, including some fields starting with "_"
kwargs_list = [
    {
        "_id": 1,  # this field is for tracking and will not be passed to get_probs
        "messages": [{"role": "user", "content": "Choose A or B?"}],
        "outputs": ["A", "B"],
        "postprocess": lambda x: x.strip()
    },
    {
        "_id": 2,  # tracking field
        "messages": [{"role": "user", "content": "Select option 1 or 2?"}],
        "outputs": ["1", "2"],
        "postprocess": None  # No postprocessing in this case
    },
]

In [ ]:
# Run get_probs concurrently on each set of parameters using get_many
results = list(runner.get_many(runner.get_probs, kwargs_list))

# Iterate through the results and print the original kwargs and the corresponding probabilities
for kwargs, prob_result in results:
    print("Input parameters:", kwargs)
    print("Probability result:", prob_result)

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:00<00:00,  3.80it/s]

Input parameters: {'_id': 1, 'messages': [{'role': 'user', 'content': 'Choose A or B?'}], 'outputs': ['A', 'B'], 'postprocess': <function <lambda> at 0x7544312e4f40>}
Probability result: {'A': 0.7879617892391346, 'B': 0.21203821076086538}
Input parameters: {'_id': 2, 'messages': [{'role': 'user', 'content': 'Select option 1 or 2?'}], 'outputs': ['1', '2'], 'postprocess': None}
Probability result: {'1': 1.0, '2': 0.0}
